In [4]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch/minsearch.py

--2025-10-07 19:43:12--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6228 (6.1K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   6.08K  --.-KB/s    in 0s      

2025-10-07 19:43:12 (60.1 MB/s) - ‘minsearch.py’ saved [6228/6228]



In [2]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [4]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [5]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

openai_client = OpenAI()

In [6]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [7]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [8]:
def llm(prompt):
    response = openai_client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [9]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [10]:
rag('how do I run kafka?')

'To run Kafka, for a Java project, you need to execute the relevant class within a JAR file in your project directory. You can do this using the command:\n\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nFor a Python Kafka project, ensure you are in a virtual environment, and then run your Python files within that environment. If you have a specific script, say `producer.py`, you first need to activate the virtual environment and ensure all dependencies are installed before running the script.'

In [17]:
rag('the course has already started, can I still enroll?')

"Yes, you can still enroll in the course after it has started. You are eligible to submit the homeworks even if you're not registered. However, be mindful of the deadlines for turning in the final projects, so it's important not to leave everything until the last minute."

## RAG with Vector Search

In [11]:
from qdrant_client import QdrantClient, models

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
qd_client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [13]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [14]:
# Define the collection name
collection_name = "zc_RAG_VecSearch"
qd_client.delete_collection(collection_name=collection_name)

# Create the collection with specified vector parameters
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [15]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword" # exact matching on string metadata fields
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [16]:
points = []
id = 0


for id_, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, 
                               model=model_handle) #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
    point = models.PointStruct(
        id=id_,
        vector=vector, 
        payload=doc
    )
    points.append(point)

In [17]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [24]:
question = "I just discovered a course, can I still join?"

In [32]:
def vector_search(question):
    print("Vector Search is used.")
    course = 'data-engineering-zoomcamp'
    
    query_pt = qd_client.query_points(
            collection_name=collection_name,
            query=models.Document( 
                text=question,
                model=model_handle 
            ),
            query_filter=models.Filter( 
                    must=[
                        models.FieldCondition(
                            key="course",
                            match=models.MatchValue(value=course)
                        )
                    ]
                ),
            limit=5, # top closest matches
            with_payload=True #to get metadata in the results
        )
    results = []
    for pt in query_pt.points:
        results.append(pt.payload)
    return results

In [33]:
vector_search('how do I run kafka?')

Vector Search is used.


[{'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
  'section': 'Module 6: streaming with kafka',
  'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'For example, when running JsonConsumer.java, got:\nConsuming form kafka started\nRESULTS:::0\nRESULTS:::0\nRESULTS:::0\nOr when running JsonProducer.java, got:\nException in thread "main" java.util.concurrent.ExecutionException: org.apache.kafka.common.errors.SaslAuthenticationException: Authentication failed\nSolution:\nMake sure in the scripts in src/main/java/org/example/ that you are running (e.g. JsonConsumer.java, JsonProducer.java), the StreamsConfig.BOOTSTRAP_SERVERS_CONFIG is the correct server url (e.g. europe-west3 from example vs europe-west2)\nMake sure cluster key and secrets are updated in src/main/java/org/example/Secrets.java (KAFKA_CLUSTER_KEY and KA

In [34]:
def rag(query):
    search_results = vector_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [35]:
rag('how do I run kafka?')

Vector Search is used.


'To run a Kafka Java producer, consumer, or streams application in the terminal, navigate to the project directory and run the following command:\n\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nReplace `<jar_name>` with the name of your JAR file. Make sure that your Kafka broker is properly set up and running, and ensure cluster keys and secrets are correctly configured.'